In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.4 MB/s eta 0:00:00


In [2]:
import os

os.environ["GROQ_API_KEY"] = "Insert_API"


In [3]:
from groq import Groq
import os, requests

# --- Must already be set: os.environ["GROQ_API_KEY"] = "..."
if "GROQ_API_KEY" not in os.environ:
    raise RuntimeError("Set GROQ_API_KEY first")

# 1) Fetch currently available models from Groq
models_url = "https://api.groq.com/openai/v1/models"
headers = {"Authorization": f"Bearer {os.environ['GROQ_API_KEY']}"}
resp = requests.get(models_url, headers=headers, timeout=30)
resp.raise_for_status()
available = [m["id"] for m in resp.json().get("data", [])]

print("✅ Available models (first 15):")
print(available[:15])

# 2) Pick best available model (prefer 70B versatile, fallback to 8B instant)
preferred_order = [
    "llama-3.3-70b-versatile",  # recommended replacement for old 70b-8192
    "llama-3.1-8b-instant",     # lightweight/faster fallback
]

MODEL = next((m for m in preferred_order if m in available), None)
if not MODEL:
    # fallback: just pick the first available model
    MODEL = available[0]

print(f"\n✅ Using model: {MODEL}")

client = Groq()

# ------------------------------------------------
# ROOT PERSONA / SYSTEM PROMPT
# ------------------------------------------------
ROOT_PERSONA = """
You are Saran Cheeku, a junior full-stack and GenAI engineer based in United States.

Tone:
- Calm, professional, strict on correctness,to the spot.

Rules:
1) Think step-by-step privately.
2) Do NOT reveal chain-of-thought.
3) Output ONLY:
   - Reasoning Summary
   - Final Answer
   - Verification Checklist

Patterns:
- Cognitive Verifier Pattern
- Question Refinement Pattern
- Provide New Information and Ask Questions Pattern
- RGC Framework (Role–Goal–Constraints)
"""

def run_prompt(title, user_prompt):
    print(f"\n{'='*80}\n🧪 {title}\n{'='*80}\n")
    completion = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": ROOT_PERSONA.strip()},
            {"role": "user", "content": user_prompt.strip()}
        ],
        temperature=0.2
    )
    print(completion.choices[0].message.content)

# ------------------------------------------------
# 1) ZERO-SHOT
# ------------------------------------------------
run_prompt(
    "ZERO-SHOT PROMPTING",
    """
Design a production-ready authentication and session management flow for a modern React + Node.js web application used by thousands of users.

Constraints:
- Use JWT-based authentication
- Clearly explain access token and refresh token lifecycle
- Identify where input validation happens and where identity verification happens
- Mention at least 3 common security risks and how they are mitigated
- End with a verification checklist

"""
)

# ------------------------------------------------
# 2) QUESTION REFINEMENT
# ------------------------------------------------
run_prompt(
    "QUESTION REFINEMENT PATTERN",
    """
Our application has user login, but we are facing security concerns and occasional unauthorized access.

Constraints:
- The problem statement is intentionally incomplete
- Apply the Question Refinement Pattern
- Ask no more than 2 essential clarifying questions
- Clearly state assumptions before proceeding
- Provide a secure login solution after refinement

"""
)

# ------------------------------------------------
# 3) ONE-SHOT
# ------------------------------------------------
run_prompt(
    "ONE-SHOT PROMPTING",
    """
Example Input:
Secure an API endpoint that is publicly accessible.

Example Output:
Reasoning Summary:
- Public endpoints require authentication, validation, and abuse protection.
Final Answer:
- JWT authentication middleware
- Request schema validation
- Rate limiting per IP
Verification Checklist:
- Unauthorized access returns 401
- Invalid payload returns 400
- Excessive requests return 429

NOW DO THIS:
Design a secure and user-friendly password reset flow for a web application that handles sensitive user data.

"""
)

# ------------------------------------------------
# 4) FEW-SHOT
# ------------------------------------------------
run_prompt(
    "FEW-SHOT PROMPTING",
    """
Example 1:
Task: Design a caching strategy for a high-traffic REST API.
Output:
- Use Redis for frequently accessed data
- Apply TTL to avoid stale cache
- Invalidate cache on update events

Example 2:
Task: Design a database schema for tracking student attendance.
Output:
- Separate users and attendance tables
- Use indexed foreign keys for fast queries

NOW DO THIS:
Design a complete Retrieval-Augmented Generation (RAG) pipeline for a company internal FAQ chatbot used by employees.

"""
)

# ------------------------------------------------
# 5) RGC FILL-IN-THE-BLANK
# ------------------------------------------------
run_prompt(
    "RGC FILL-IN-THE-BLANK",
    """
Fill the RGC template for the following task:
"Design a rate-limited FastAPI endpoint for handling chat messages in a public-facing application."

RGC TEMPLATE:
[R] ROLE:
[G] GOAL:
[C] CONSTRAINTS:
- Format:
- Must include:
- Must avoid:
- Verification:

"""
)

print("\n✅ Prompt Engineering Activity Completed Successfully")


✅ Available models (first 15):
['openai/gpt-oss-120b', 'moonshotai/kimi-k2-instruct-0905', 'meta-llama/llama-prompt-guard-2-22m', 'meta-llama/llama-prompt-guard-2-86m', 'groq/compound-mini', 'openai/gpt-oss-safeguard-20b', 'qwen/qwen3-32b', 'llama-3.1-8b-instant', 'meta-llama/llama-guard-4-12b', 'meta-llama/llama-4-maverick-17b-128e-instruct', 'groq/compound', 'allam-2-7b', 'llama-3.3-70b-versatile', 'whisper-large-v3-turbo', 'meta-llama/llama-4-scout-17b-16e-instruct']

✅ Using model: llama-3.3-70b-versatile

🧪 ZERO-SHOT PROMPTING

## Reasoning Summary
To design a production-ready authentication and session management flow for a modern React + Node.js web application, we will utilize JWT-based authentication. The flow will involve the following steps:
- User registration: The user will provide their details, which will be validated and then stored in the database.
- User login: The user will provide their credentials, which will be verified, and if correct, a JWT access token and refr